# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [227]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [228]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** 

Качество не изменится, но в полученой матрице количество признаков будет равно количеству строк, что значительно увеличит время обучения модели.


**Обоснование:** 

Предположим, что качество не меняется. Значит, предсказания равны друг другу:

$$
a = a_P
$$
$$
Xw = XPw_P
$$
$$
X((X^T X)^{-1} X^T y) = XP(((XP)^T XP)^{-1} (XP)^T y)
$$
$$
(X^T X)^{-1} X^T y = P((P^TX^T XP)^{-1} P^T X^T y)
$$
$$
(X^T X)^{-1} X^T y = P((X^TXP)^{-1} P^{T-1} P^T X^T y)
$$
$$
(X^T X)^{-1} X^T y = P(X^TXP)^{-1} X^T y
$$
$$
(X^T X)^{-1} X^T y = PP^{-1}(X^TX)^{-1} X^T y
$$
$$
(X^T X)^{-1} X^T y = (X^TX)^{-1} X^T y
$$
$$
I = I
$$

## Алгоритм преобразования

**Алгоритм**
$$
X^P = (X + p_1) \cdot p_2
$$
$p_1, p_2$ - векторы случайных чисел $>0$.

**Обоснование**

Матрица признаков умножается на вектор случаных чисел > 0. Для избежения нулевых значений, перед умножением к матрице прибавляется другой вектор без нулевых значений.

При прибавлении или умножении на вектор все значения признака изменяются на одну и ту же величину, что минимизирует влияние на результат линейной регресии.

## Проверка алгоритма

In [229]:
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [230]:
X.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


In [231]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
score = r2_score(y_test, predictions)
score

0.41541402565474894

In [232]:
np.random.seed(42)
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']
p1 =  np.random.uniform(low=1, high=1000, size=X.shape[1])
p2 =  np.random.uniform(low=1, high=1000, size=X.shape[1])

X = (X + p1) * p2 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [233]:
X.head()

,Пол,Возраст,Зарплата,Члены семьи
0,59006.318911,155546.739743,2.970888e+06,519837.809389
1,58849.456289,156330.932372,2.286192e+06,519837.809389
2,58849.456289,153664.677434,1.282758e+06,518971.499419
3,58849.456289,152409.969227,2.504587e+06,520704.119359
4,59006.318911,153507.838908,1.583788e+06,518971.499419


После преобразования сложно понять смысл значений и получить из них персональные данные.

In [234]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
p_score = r2_score(y_test, predictions)
p_score

0.4154140256547487

In [235]:
score - p_score

2.220446049250313e-16

**Вывод**

Разница между качеством моделей минимальная.